In [75]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 1000)
from bokeh.models import Panel, Tabs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import lightgbm as lgb
import plotly.figure_factory as ff
import gc
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import json
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')



path = './tensorflow2-question-answering/'
train_path = 'simplified-nq-train.jsonl'
test_path = 'simplified-nq-test.jsonl'
sample_submission_path = 'sample_submission.csv'


def read_data(path, sample = True, chunksize = 30000):
    if sample == True:
        df = []
        with open(path, 'rt') as reader:
            for i in range(chunksize):
                df.append(json.loads(reader.readline()))
        df = pd.DataFrame(df)
        print('Our sampled dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    else:
        df = pd.read_json(path, orient = 'records', lines = True)
        print('Our dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
        gc.collect()
    return df

train = read_data(path+train_path, sample = True)
test = read_data(path+test_path, sample = False)
print("Train data is")
train.head(10)






Our sampled dataset have 30000 rows and 6 columns
Our dataset have 346 rows and 4 columns
Train data is


,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018
5,List of regions of Africa - wikipedia <H1> Lis...,"[{'start_token': 14, 'top_level': True, 'end_t...",what are the different regions of africa and h...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,3411244446249504947
6,Pom Klementieff - wikipedia <H1> Pom Klementie...,"[{'start_token': 14, 'top_level': True, 'end_t...",who played mantis guardians of the galaxy 2,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Po...,-2500044561429484630
7,Frosty the Snowman ( film ) - wikipedia <H1> F...,"[{'start_token': 22, 'top_level': True, 'end_t...",who did the voice of the magician in frosty th...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Fr...,5611750702541347162
8,History of the Acadians - wikipedia <H1> Histo...,"[{'start_token': 23, 'top_level': True, 'end_t...",what indian tribe did the acadians form friend...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hi...,4958098854057393062
9,Outer Banks - wikipedia <H1> Outer Banks </H1>...,"[{'start_token': 59, 'top_level': True, 'end_t...",what is considered the outer banks in north ca...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ou...,8796576945844451825


In [80]:
print(train['document_url'][0])

https://en.wikipedia.org//w/index.php?title=Email_marketing&amp;oldid=814071202


In [79]:
test = read_data(path+test_path, sample = False)
print("Test data is")
test.head(10)


Our dataset have 346 rows and 4 columns
Test data is


,example_id,question_text,document_text,long_answer_candidates
0,-1220107454853145600,who is the south african high commissioner in ...,"High Commission of South Africa , London - wik...","[{'end_token': 136, 'start_token': 18, 'top_le..."
1,8777415633185303552,the office episode when they sing to michael,Michael 's Last Dundies - wikipedia <H1> Micha...,"[{'end_token': 190, 'start_token': 23, 'top_le..."
2,4640548859154538496,what is the main idea of the cross of gold speech,Cross of gold speech - wikipedia <H1> Cross of...,"[{'end_token': 165, 'start_token': 12, 'top_le..."
3,-5316095317154496512,when was i want to sing in opera written,Wilkie Bard - wikipedia <H1> Wilkie Bard </H1>...,"[{'end_token': 105, 'start_token': 8, 'top_lev..."
4,-8752372642178983936,who does the voices in ice age collision course,Ice Age : Collision Course - Wikipedia <H1> Ic...,"[{'end_token': 287, 'start_token': 16, 'top_le..."
5,-3401345000915256320,who are the owners of the carolina hurricanes,Carolina Hurricanes - Wikipedia <H1> Carolina ...,"[{'end_token': 317, 'start_token': 38, 'top_le..."
6,4774513718149378048,what are the minds two tracks and what is dual...,Dual process theory - wikipedia <H1> Dual proc...,"[{'end_token': 149, 'start_token': 19, 'top_le..."
7,5354793787345685504,red cross got how many times nobel prize,List of Nobel laureates - wikipedia <H1> List ...,"[{'end_token': 218, 'start_token': 69, 'top_le..."
8,-430548376093792256,when did they finish building the sydney opera...,Sydney Opera House - Wikipedia <H1> Sydney Ope...,"[{'end_token': 722, 'start_token': 23, 'top_le..."
9,1166366462619398656,where was the first sample of ascorbic acid is...,Chemistry of ascorbic acid - wikipedia <H1> Ch...,"[{'end_token': 698, 'start_token': 35, 'top_le..."


In [81]:
sample_submission = pd.read_csv(path + sample_submission_path)
print('Our sample submission have {} rows'.format(sample_submission.shape[0]))
sample_submission.head(10)

Our sample submission have 692 rows


,example_id,PredictionString
0,-1011141123527297803_long,NaN
1,-1011141123527297803_short,NaN
2,-1028916936938579349_long,NaN
3,-1028916936938579349_short,NaN
4,-1055197305756217938_long,NaN
5,-1055197305756217938_short,NaN
6,-1074129516932871805_long,NaN
7,-1074129516932871805_short,NaN
8,-1114334749483663139_long,NaN
9,-1114334749483663139_short,NaN


In [82]:
# Lets see if we have missing values 

def missing_values(df):
    df = pd.DataFrame(df.isnull().sum()).reset_index()
    df.columns = ['features', 'n_missing_values']
    return df
    



In [35]:
missing_values(train)

,features,n_missing_values
0,document_text,0
1,long_answer_candidates,0
2,question_text,0
3,annotations,0
4,document_url,0
5,example_id,0


In [83]:
missing_values(test)
# cool we dont have missing values

,features,n_missing_values
0,example_id,0
1,question_text,0
2,document_text,0
3,long_answer_candidates,0


In [85]:
#let's explore the first line of our train set to understand the logic of this dataset.
question_text_0 = train.loc[0,'question_text']
question_text_0

'which is the most common use of opt-in e-mail marketing'

In [86]:
# this is the main quetion we need to answer 

document_text_0 = train.loc[0,'document_text'].split()
" ".join(document_text_0[:800])

"Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <T

In [87]:
# So in the first column we have a huge wikipedia text. This is where we need to find the answer for the previous question

long_answer_candidates_0 = train.loc[0,'long_answer_candidates']
long_answer_candidates_0[0:10]


[{'start_token': 14, 'top_level': True, 'end_token': 170},
 {'start_token': 15, 'top_level': False, 'end_token': 169},
 {'start_token': 52, 'top_level': False, 'end_token': 103},
 {'start_token': 53, 'top_level': False, 'end_token': 102},
 {'start_token': 103, 'top_level': False, 'end_token': 156},
 {'start_token': 104, 'top_level': False, 'end_token': 155},
 {'start_token': 170, 'top_level': True, 'end_token': 321},
 {'start_token': 171, 'top_level': False, 'end_token': 180},
 {'start_token': 180, 'top_level': False, 'end_token': 186},
 {'start_token': 186, 'top_level': False, 'end_token': 224}]

In [88]:
#This are all the possibles long answers ranges. In other words they give you the start indices and last indices of all the possibles long answers in the document text columns that could answer the question.

annotations_0 = train['annotations'][0][0]
annotations_0


{'yes_no_answer': 'NONE',
 'long_answer': {'start_token': 1952,
  'candidate_index': 54,
  'end_token': 2019},
 'short_answers': [{'start_token': 1960, 'end_token': 1969}],
 'annotation_id': 593165450220027640}

In [89]:
#Lets check the entire logic of the first line of our train set

print('Our question is : ', question_text_0)
print('Our short answer is : ', " ".join(document_text_0[annotations_0['short_answers'][0]['start_token']:annotations_0['short_answers'][0]['end_token']]))
print('Our long answer is : ', " ".join(document_text_0[annotations_0['long_answer']['start_token']:annotations_0['long_answer']['end_token']]))

Our question is :  which is the most common use of opt-in e-mail marketing
Our short answer is :  a newsletter sent to an advertising firm 's customers
Our long answer is :  <P> A common example of permission marketing is a newsletter sent to an advertising firm 's customers . Such newsletters inform customers of upcoming events or promotions , or new products . In this type of advertising , a company that wants to send a newsletter to their customers may ask them at the point of purchase if they would like to receive the newsletter . </P>


In [90]:
# lets collect yes/ no answer 

yes_no_answer = []
for i in range(len(train)):
    yes_no_answer.append(train['annotations'][i][0]['yes_no_answer'])
yes_no_answer = pd.DataFrame({'yes_no_answer': yes_no_answer})

yes_no_answer


# plot a bar 

def bar_plot(df, column, title, width, height, n, get_count = True):
    if get_count == True:
        cnt_srs = df[column].value_counts(normalize = True)[:n]
    else:
        cnt_srs = df
        
    trace = go.Bar(
        x = cnt_srs.index,
        y = cnt_srs.values,
        marker = dict(
            color = '#1E90FF',
        ),
    )

    layout = go.Layout(
        title = go.layout.Title(
            text = title,
            x = 0.5
        ),
        font = dict(size = 14),
        width = width,
        height = height,
    )

    data = [trace]
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig, filename = 'bar_plot')
bar_plot(yes_no_answer, 'yes_no_answer', 'Yes No Answer Distribution', 800, 500, 3)

In [91]:
# this function extract the short answers and fill a dataframe
def extract_target_variable(df, short = True):
    if short:
        short_answer = []
        for i in range(len(df)):
            short = df['annotations'][i][0]['short_answers']
            if short == []:
                yes_no = df['annotations'][i][0]['yes_no_answer']
                if yes_no == 'NO' or yes_no == 'YES':
                    short_answer.append(yes_no)
                else:
                    short_answer.append('EMPTY')
            else:
                short = short[0]
                st = short['start_token']
                et = short['end_token']
                short_answer.append(f'{st}'+':'+f'{et}')
        short_answer = pd.DataFrame({'short_answer': short_answer})
        return short_answer
    else:
        long_answer = []
        for i in range(len(df)):
            long = df['annotations'][i][0]['long_answer']
            if long['start_token'] == -1:
                long_answer.append('EMPTY')
            else:
                st = long['start_token']
                et = long['end_token']
                long_answer.append(f'{st}'+':'+f'{et}')
        long_answer = pd.DataFrame({'long_answer': long_answer})
        return long_answer
        
short_answer = extract_target_variable(train)
short_answer.head()

,short_answer
0,1960:1969
1,213:215
2,EMPTY
3,512:514
4,EMPTY


In [60]:
short_answer['type'] = short_answer['short_answer'].copy()
short_answer.loc[(short_answer['short_answer']!='EMPTY') & (short_answer['short_answer']!='YES') & (short_answer['short_answer']!='NO'), 'type'] =  'TEXT'
bar_plot(short_answer, 'type', 'Short Answer Distribution', 800, 500, 10)

In [92]:
long_answer = extract_target_variable(train, False)
long_answer.head()

,long_answer
0,1952:2019
1,212:310
2,319:438
3,509:576
4,EMPTY


In [93]:
long_answer['type'] = long_answer['long_answer'].copy()
long_answer.loc[(long_answer['long_answer']!='EMPTY'), 'type'] =  'TEXT'
bar_plot(long_answer, 'type', 'Long Answer Distribution', 800, 500, 10)

In [64]:
def count_word_frequency(series, top = 0, bot = 20):
    cv = CountVectorizer()   
    cv_fit = cv.fit_transform(series)    
    word_list = cv.get_feature_names(); 
    count_list = cv_fit.toarray().sum(axis=0)
    frequency = pd.DataFrame({'Word': word_list, 'Frequency': count_list})
    frequency.sort_values(['Frequency'], ascending = False, inplace = True)
    frequency['Percentage'] = frequency['Frequency']/frequency['Frequency'].sum()
    frequency.drop('Frequency', inplace = True, axis = 1)
    frequency['Percentage'] = frequency['Percentage'].round(3)
    frequency = frequency.iloc[top:bot]
    frequency.set_index('Word', inplace = True)
    bar_plot(pd.Series(frequency['Percentage']), 'Percentage', 'Question Text Word Frequency Distribution', 800, 500, 20, False)
    return frequency
    
frequency = count_word_frequency(train['question_text'])

In [94]:
frequency = count_word_frequency(train['question_text'], 20, 40)


In [95]:
frequency = count_word_frequency(test['question_text'])

In [96]:
#Top words are repeated
frequency = count_word_frequency(test['question_text'], 20, 40)

In [97]:
frequency = count_word_frequency(test['document_text'])

In [71]:
def build_train_test_long(df, train = True):
    final_long_answer_frame = pd.DataFrame()
    if train == True:
        # get long answer
        long_answer = extract_target_variable(df, False)
        
        # iterate over each row to get the possible answers
        for index, row in df.iterrows():
            start_end_tokens = []
            questions = []
            responds = []
            for i in row['long_answer_candidates']:
                start_token = i['start_token']
                end_token = i['end_token']
                start_end_token = str(i['start_token']) + ':' + str(i['end_token'])
                question = row['question_text']
                respond = " ".join(row['document_text'].split()[start_token : end_token])
                start_end_tokens.append(start_end_token)
                questions.append(question)
                responds.append(respond)

            long_answer_frame = pd.DataFrame({'question': questions, 'respond': responds, 'start_end_token': start_end_tokens})
            long_answer_frame['answer'] = long_answer.iloc[index][0]
            long_answer_frame['target'] = long_answer_frame['start_end_token'] == long_answer_frame['answer']
            long_answer_frame['target'] = long_answer_frame['target'].astype('int16')
            long_answer_frame.drop(['answer'], inplace = True, axis = 1)
            final_long_answer_frame = pd.concat([final_long_answer_frame, long_answer_frame])
        return final_long_answer_frame
    else:
         # iterate over each row to get the possible answers
        for index, row in df.iterrows():
            start_end_tokens = []
            questions = []
            responds = []
            for i in row['long_answer_candidates']:
                start_token = i['start_token']
                end_token = i['end_token']
                start_end_token = str(i['start_token']) + ':' + str(i['end_token'])
                question = row['question_text']
                respond = " ".join(row['document_text'].split()[start_token : end_token])
                start_end_tokens.append(start_end_token)
                questions.append(question)
                responds.append(respond)

            long_answer_frame = pd.DataFrame({'question': questions, 'respond': responds, 'start_end_token': start_end_tokens})
            final_long_answer_frame = pd.concat([final_long_answer_frame, long_answer_frame])
        return final_long_answer_frame
        


def build_train_test_short(df, train = True):
    
    final_short_answer_frame = pd.DataFrame()
    
    if train == True:
        # get short answer
        short_answer = extract_target_variable(df, True)

        # iterate over each row to get the possible answer
        for index, row in df.iterrows():
            start_tokens = []
            end_tokens = []
            start_end_tokens = []
            questions = []
            responds = []
            for i in row['long_answer_candidates']:
                start_token = i['start_token']
                end_token = i['end_token']
                start_end_token = str(i['start_token']) + ':' + str(i['end_token'])
                question = row['question_text']
                respond = " ".join(row['document_text'].split()[int(start_token) : int(end_token)])
                start_tokens.append(start_token)
                end_tokens.append(end_token)
                start_end_tokens.append(start_end_token)
                questions.append(question)
                responds.append(respond)

            short_answer_frame = pd.DataFrame({'question': questions, 'respond': responds, 'start_token': start_tokens, 'end_token': end_tokens, 'start_end_token': start_end_tokens})
            short_answer_frame['answer'] = short_answer.iloc[index][0]
            short_answer_frame['start_token_an'] = short_answer_frame['answer'].apply(lambda x: x.split(':')[0] if ':' in x else 0)
            short_answer_frame['end_token_an'] = short_answer_frame['answer'].apply(lambda x: x.split(':')[1] if ':' in x else 0)
            short_answer_frame['start_token_an'] = short_answer_frame['start_token_an'].astype(int)
            short_answer_frame['end_token_an'] = short_answer_frame['end_token_an'].astype(int)
            short_answer_frame['target'] = 0
            short_answer_frame.loc[(short_answer_frame['start_token_an'] >= short_answer_frame['start_token']) & (short_answer_frame['end_token_an'] <= short_answer_frame['end_token']), 'target'] = 1
            short_answer_frame.drop(['answer', 'start_token', 'end_token', 'start_token_an', 'end_token_an'], inplace = True, axis = 1)
            final_short_answer_frame = pd.concat([final_short_answer_frame, short_answer_frame])
        return final_short_answer_frame
    else:
        # iterate over each row to get the possible answer
        for index, row in df.iterrows():
            start_end_tokens = []
            questions = []
            responds = []
            for i in row['long_answer_candidates']:
                start_token = i['start_token']
                end_token = i['end_token']
                start_end_token = str(i['start_token']) + ':' + str(i['end_token'])
                question = row['question_text']
                respond = " ".join(row['document_text'].split()[int(start_token) : int(end_token)])
                start_end_tokens.append(start_end_token)
                questions.append(question)
                responds.append(respond)

            short_answer_frame = pd.DataFrame({'question': questions, 'respond': responds, 'start_end_token': start_end_tokens})
            final_short_answer_frame = pd.concat([final_short_answer_frame, short_answer_frame])
        return final_short_answer_frame

In [72]:
sh = build_train_test_long(train.head())
sh.head()

,question,respond,start_end_token,target
0,which is the most common use of opt-in e-mail ...,<Table> <Tr> <Td> </Td> <Td> ( hide ) This art...,14:170,0
1,which is the most common use of opt-in e-mail ...,<Tr> <Td> </Td> <Td> ( hide ) This article has...,15:169,0
2,which is the most common use of opt-in e-mail ...,<Table> <Tr> <Td> </Td> <Td> This article need...,52:103,0
3,which is the most common use of opt-in e-mail ...,<Tr> <Td> </Td> <Td> This article needs additi...,53:102,0
4,which is the most common use of opt-in e-mail ...,<Table> <Tr> <Td> </Td> <Td> This article poss...,103:156,0


In [98]:
sh[sh['target']==1]



,question,respond,start_end_token,target
54,which is the most common use of opt-in e-mail ...,<P> A common example of permission marketing i...,1952:2019,1
15,how i.met your mother who is the mother,"<P> Tracy McConnell , better known as `` The M...",212:310,1
24,what type of fertilisation takes place in humans,<P> The process of fertilization involves a sp...,319:438,1
59,who had the most wins in the nfl,<P> Active quarterback Tom Brady holds the rec...,509:576,1
